# TME 2 

## 1. Introduction

In [1]:
import pandas as pd
import itertools as it
import numpy as np

In [2]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M2 = [[0,2,3,8,14,18],[2,0,3,8,14,18],
      [3,3,0,8,14,18],[8,8,8,0,14,18],
      [14,14,14,14,0,18],[18,18,18,18,18,0]]
#UPGMA
M3 = [[0,19,27,8,33,18,13],[19,0,31,18,36,1,13],
          [27,31,0,26,41,32,29],[8,18,26,0,31,17,14],
          [33,36,41,31,0,35,28],[18,1,32,17,35,0,12],
          [13,13,29,14,28,12,0]]
#Neighbor Joining
M4 = [[0,2,4,6,6,8],[2,0,4,6,6,8],
          [4,4,0,6,6,8],[6,6,6,0,4,8],
          [6,6,6,4,0,8],[8,8,8,8,8,0]]

M1D = pd.DataFrame(M1)
M2D = pd.DataFrame(M2)
UPGMAD = pd.DataFrame(M3) 
NJD = pd.DataFrame(M4)

M1D

,0,1,2,3
0,0,8,7,12
1,8,0,9,14
2,7,9,0,11
3,12,14,11,0


In [3]:
def convert(M):
    M = np.array(M).astype(float)
    il1 = np.triu_indices(len(M))
    M[il1] = np.nan
    MD = pd.DataFrame(M)
    return MD

convert(M1)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,8.0,NaN,NaN,NaN
2,7.0,9.0,NaN,NaN
3,12.0,14.0,11.0,NaN


###### Question 1

##### Matrice additive

Une est matrice additive si et seulement si la condition des 4 points est vraie pour tous les quartets i,j,k et l où 1<=i,j,k,l<=n).
La condition des 4 points est remplie quand Dij + Dkl <= max(Dik+Djl,Dil+Djk).

In [4]:
def estadditive(M):
    comb=it.combinations(range(len(M)),4)
    for groupe in comb:
        i,j,k,l=groupe
        if not (M.at[i,j]+M.at[k,l]<=max(M.at[i,k]+M.at[j,l],M.at[i,l]+M.at[j,k])):
            return False
    return True

In [5]:
print(M1D,"\n",estadditive(M1D))

    0   1   2   3
0   0   8   7  12
1   8   0   9  14
2   7   9   0  11
3  12  14  11   0 
 True


##### Ultramétricité

Une matrice est dite ultramétrique quand, pour tout choix de 3 points i,j,k au moins deux distances sont égales : Dij et Djk.

On a alors mathématiquement : Dij = Djk >Dik

Une matrice ultramétrique est forcément additive.

In [6]:
def estultra(M):
    comb=it.combinations(range(len(M)),3)
    for groupe in comb:
        i,j,k=groupe
        if not(M.at[i,k] <= max(M.at[i,j],M.at[j,k])):
            return False
    return True

In [7]:
print(M2D,"\n", estultra(M2D))

    0   1   2   3   4   5
0   0   2   3   8  14  18
1   2   0   3   8  14  18
2   3   3   0   8  14  18
3   8   8   8   0  14  18
4  14  14  14  14   0  18
5  18  18  18  18  18   0 
 True


###### Question 2

In [8]:
#Calcul de la somme des distances d'un cluster avec les autres
def cluster(M,i):            #i = l'espèce en question
    return sum(M.iloc[i])

In [9]:
#Calcul de toutes les sommes de distances entre les différentes clusters 
def groupe(M):
    L=[]
    for k in range(len(M)):
        L.append(cluster(M,k))
    return L


In [10]:
def group_clust_df(M):
    return M.sum()

In [11]:
cluster(M1D,1)
groupe(M1D)

group_clust_df(M1D)

0    27
1    31
2    27
3    37
dtype: int64

## 2 . UPGMA

###### Question 1 

Le format Newick est un format de fichier utilisé en phylogénie pour décrire les relations phylogénétiques entre différents clusters.
Par exemple si on a un ancêtre commun entre A et B puis entre AB et C on écrira sous ce format :

(C : longueur de la branche (A : longueur, B : longueur) : longueur restante de la branche)

L'extension de ce format est .nwk

In [12]:
UPGMAD

,0,1,2,3,4,5,6
0,0,19,27,8,33,18,13
1,19,0,31,18,36,1,13
2,27,31,0,26,41,32,29
3,8,18,26,0,31,17,14
4,33,36,41,31,0,35,28
5,18,1,32,17,35,0,12
6,13,13,29,14,28,12,0


In [13]:
def UPGMA(M):
    
    UPGMAD2 = convert(M3)
    UPGMAD3 = UPGMAD2.copy()
    
    L = []
    
    for i in range(len(UPGMAD2)):
        idx = np.unravel_index(np.nanargmin(UPGMAD3.values), UPGMAD3.shape)
        
        idxmi = min(idx)
        idxma = max(idx)
        
        L.append(UPGMAD3.at[idx]/2)
        
        UPGMAD3.at[idxmi,0] = (UPGMAD2.at[idxmi,0] + UPGMAD2.at[idxma,0])/2 
        for j in range(len(UPGMAD3)):
            if(j<idxma):
                UPGMAD3.at[j,idxmi] =  (UPGMAD2.at[j,idxmi] + UPGMAD2.at[idxma,j])/2 
            if(j>idxma):
                UPGMAD3.at[j,idxmi] =  (UPGMAD2.at[j,idxmi] + UPGMAD2.at[j,idxma])/2

        
        UPGMAD3 = UPGMAD3.rename(index = {idxmi: str(idxmi)+"-"+str(idxma)}, columns = {idxmi: str(idxmi)+"-"+str(idxma)})
        UPGMAD3 = UPGMAD3.drop(index = idxma, columns = idxma)

    return L

In [14]:
#S = UPGMA(M3)
#S

UPGMAD.shape == (7,7)

UPGMAD.idxmin()

a = convert(UPGMAD)

b = a.stack().idxmin()

a.drop(b[1], axis=1, inplace=True)
a.drop(b[0], axis=0, inplace=True)

new_row = np.zeros(len(a))
new_row[:] = np.nan 

a.loc[str(b)] = new_row



#a = pd.concat([a, pd.DataFrame(new_row)], axis=1, ignore_index=True)



a.insert(loc=0,   column= str(b),   value = np.NaN )


#a.append(pd.Series(), ignore_index=True)




a.rename({str(len(a)-1): str(b)}, axis='index')

a.columns[0]


'(5, 1)'

In [15]:
def UPGMA(M):
    
    matrix_UPGMA = M.copy()
    nwk = ""
    
    while(matrix_UPGMA.shape != (1,1)):       
        d_min = matrix_UPGMA.stack().idxmin()
        
        new_row = np.zeros(len(a))
        new_row[:] = np.nan 
        
        new_matrix_UPGMA = matrix_UPGMA.copy()

        new_matrix_UPGMA.loc[str(d_min)] = new_row
        new_matrix_UPGMA.insert(loc=0, column= str(d_min), value = np.NaN)
        new_matrix_UPGMA.drop(d_min[0], axis=0, inplace=True)
        new_matrix_UPGMA.drop(d_min[1], axis=1, inplace=True)
        
        for i in new_matrix_UPGMA.columns:
            new_matrix_UPGMA.at[str(d_min),i] = new_matrix_UPGMA.at[i,str(d_min)] = 
    

SyntaxError: invalid syntax (295974441.py, line 20)

In [16]:
#UPGMAD2 : copie float
#UPGMAD3 : modifiable

UPGMAD2 = convert(M3)
UPGMAD2
UPGMAD3 = UPGMAD2.copy()
UPGMAD3

#liste avec longueur branches : 
dico = {}

In [17]:
#Index minimal trouvé
idx = np.unravel_index(np.nanargmin(UPGMAD3.values), UPGMAD3.shape)

print(idx)
idxmi = min(idx)
idxma = max(idx)

dico[idx] = UPGMAD3.at[idx]/2



UPGMAD3.at[idxmi,0] = (UPGMAD2.at[idxmi,0] + UPGMAD2.at[idxma,0])/2 
for i in range(len(UPGMAD3)):
    if(i<idxma and i>idxmi-1):
        UPGMAD3.at[i,idxmi] =  (UPGMAD2.at[i,idxmi] + UPGMAD2.at[idxma,i])/2 
    if(i>idxma):
        UPGMAD3.at[i,idxmi] =  (UPGMAD2.at[i,idxmi] + UPGMAD2.at[i,idxma])/2

UPGMAD3 = UPGMAD3.drop(index = idxma, columns = idxma)
UPGMAD3 = UPGMAD3.rename(index = {idxmi: str(idxmi)+"-"+str(idxma)}, columns = {idxmi: str(idxmi)+"-"+str(idxma)})



print(dico)

UPGMAD3



(5, 1)
{(5, 1): 0.5}


,0,1-5,2,3,4,6
0,NaN,NaN,NaN,NaN,NaN,NaN
1-5,18.5,NaN,NaN,NaN,NaN,NaN
2,27.0,31.5,NaN,NaN,NaN,NaN
3,8.0,17.5,26.0,NaN,NaN,NaN
4,33.0,35.5,41.0,31.0,NaN,NaN
6,13.0,12.5,29.0,14.0,28.0,NaN


http://www.slimsuite.unsw.edu.au/teaching/upgma/

In [83]:
a = 3
b = str(a)
b = 0
a

3